In [ ]:
import theano 
from theano import tensor as T 
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.cm as cm 
from theano.compile.nanguardmode import NanGuardMode

srng = RandomStreams()

def dropout(X, p=0.): 
    if p > 0: 
        retain_prob = 1 - p 
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX) 
        X /= retain_prob 
    return X 

def rectify(X): 
    return T.maximum(X, 0.) 

def softmax(X):     
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x')) 
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x') 

def log_softmax(q):
    max_q = max(0.0, np.max(q))
    rebased_q = q - max_q
    return rebased_q - np.logaddexp(-max_q, np.logaddexp.reduce(rebased_q))

def floatX(X): 
    return np.asarray(X, dtype=theano.config.floatX) 

def init_weights(shape): 
    return theano.shared(floatX(np.random.randn(*shape) * 0.01)) 

def RMSprop(cost, params, lr=0.00001, rho=0.9, epsilon=1e-6): 
    grads = T.grad(cost=cost, wrt=params) 
    updates = [] 
    for p, g in zip(params, grads): 
        acc = theano.shared(p.get_value() * 0.) 
        acc_new = rho * acc + (1 - rho) * g ** 2 
        gradient_scaling = T.sqrt(acc_new + epsilon) 
        g = g / gradient_scaling 
        updates.append((acc, acc_new)) 
        updates.append((p, p - lr * g)) 
    return updates 

w_h1 = init_weights((2500, 400)) 
w_h2 = init_weights((400, 400)) 
w_h3 = init_weights((400, 7500)) 

def model(X,w_h1,w_h2,w_h3,p_drop_input=0.0, p_drop_hidden=0.0):
    X = dropout(X, p_drop_input) 
    w_o_h1=rectify(T.dot(X, w_h1)) 
    w_o_h1= dropout(w_o_h1, p_drop_input) 
    w_o_h2=rectify(T.dot(w_o_h1, w_h2)) 
    w_o_h2= dropout(w_o_h2, p_drop_input) 
    y_x = softmax(T.dot(w_o_h2, w_h3)) 
    return y_x 


X = T.fmatrix() 
Y = T.fmatrix() 

y_x = model(X,w_h1,w_h2,w_h3,0.5,0.5)

cost = T.mean(T.nnet.categorical_crossentropy(y_x, Y))
params = [w_h1,w_h2,w_h3]
updates = RMSprop(cost, params, lr=0.0001) 
train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True) 
predict = theano.function(inputs=[X], outputs=y_x, mode=NanGuardMode(nan_is_error=True, inf_is_error=True, big_is_error=True),allow_input_downcast=True) 

import PIL
from PIL import Image
import requests
from io import BytesIO

images = dict()

for i in range(1000):
    with open("../Environment/Informations/Content/Portraits/trainImages/images.txt") as f:
        infoFileContents = f.readlines()
        for l in range(len(infoFileContents)):
            infoLine = infoFileContents[l].lstrip().rstrip()
            
            trainImg = ''
            if infoLine in images:
                trainImg = images[infoLine]
            else:
                response = requests.get(infoLine)
                images[infoLine] = Image.open(BytesIO(response.content))
                trainImg = images[infoLine]
                
            trainInpImg = trainImg 
            trainInpGrayImg = trainInpImg.convert("L")
            trainInpGrayImg = trainInpGrayImg.resize((50,50), PIL.Image.ANTIALIAS)
            trainInpGreyImgArray = np.asarray(trainInpGrayImg, dtype=np.uint8).ravel()

            #plt.imshow(trainInpGrayImg)
            #plt.show()

            trainOutImg = trainImg
            trainOutImg = trainOutImg.convert("RGB")
            trainOutImg = trainOutImg.resize((50,50), PIL.Image.ANTIALIAS)
            
            #plt.imshow(trainOutImg)
            #plt.show()

            
            trainOut3dArray = np.asarray(trainOutImg, dtype=np.uint8)
            trainOut3dArrayClone = np.empty_like (trainOut3dArray)
            trainOut3dArrayClone[:] = trainOut3dArray
 
            #trainOut3dArrayClone[:,:,1:3] = 0
            trainOutImgArray  = np.rollaxis(trainOut3dArrayClone, 2)
            #print (trainOutImgArray.shape)
            #print (trainOutImgArray)
            #trainOutImgArray = trainOutImgArray.ravel()[0:2500]
            trainOutImgArray = trainOutImgArray.ravel()
            #print (trainOutImgArray)
         
            """
            predImg2dArray = trainOutImgArray.reshape((3,50,50)).astype('uint8')    
            predImg2dArray = np.rollaxis(predImg2dArray, 2)
            predImg2dArray = np.rollaxis(predImg2dArray, 2)
            img = Image.fromarray(predImg2dArray)
            plt.imshow(img)
            plt.show()
            """

            #print (trainOutImgArray)
            
            #print (trainInpGreyImgArray)
            #print (trainOutImgArray)
            cost = train([trainInpGreyImgArray], [trainOutImgArray])

with open("../Environment/Informations/Content/Portraits//testImages/images.txt") as f:
    infoFileContents = f.readlines()
    for l in range(len(infoFileContents)):
        infoLine = infoFileContents[l].lstrip().rstrip()
        response = requests.get(infoLine)
        
        print (infoLine)
        trainInpImg = Image.open(BytesIO(response.content))
        trainInpGrayImg = trainInpImg.convert("L")
        trainInpGrayImg = trainInpGrayImg.resize((50,50), PIL.Image.ANTIALIAS)
        trainInpGreyImgArray = np.asarray(trainInpGrayImg, dtype=np.uint8).ravel()
        
        plt.imshow(trainInpGrayImg, cmap = cm.Greys_r)
        plt.show()
        
        predImgArray = predict([trainInpGreyImgArray])
        predImgArray = predImgArray*1000000

        predImgArray = predImgArray.reshape((3,50,50)).astype('uint8')    
        predImgArray = np.rollaxis(predImgArray, 2)
        predImgArray = np.rollaxis(predImgArray, 2)
            
        #print (predImgArray)
        #predImg3dArray = predImgArray.reshape((50,50)).astype('uint8')
        #predImg3dArray = np.rollaxis(predImg3dArray, 2)
        #predImg3dArray = np.rollaxis(predImg3dArray, 2)
                
        img = Image.fromarray(predImgArray)

        plt.imshow(img)
        plt.show()
